In [49]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, size
import numpy as np
from datetime import datetime

VBox()

In [50]:
starttime = datetime.now() #inorder to find the execution time we store the time in starttime

VBox()

In [51]:
spark = SparkSession \
    .builder \
    .appName("Assignment 2 COMP5349 Stage_One") \
    .getOrCreate()

VBox()

In [52]:
awsData = "s3://amazon-reviews-pds/tsv/amazon_reviews_us_Music_v1_00.tsv.gz"
# read the music data from the aws

VBox()

In [53]:
musicData = spark.read.csv(awsData,header=True,sep='\t')
musicData.show(1)
#read the data to the spark in csv format and seperate each coloumn with '\t' as seperator

VBox()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+---------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+---------------+--------------------+-----------+
|         US|   10140119|R3LI5TRP3YIDQL|B00TXH4OLC|     384427924|Whatever's for Us...|           Music|          5|            0|          0|   N|                Y|     Five Stars|Love this CD alon...| 2015-08-31|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+----------

In [54]:
musicData.printSchema()
#to check the schema of the the original dataset

VBox()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: string (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: string (nullable = true)
 |-- total_votes: string (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)

In [55]:
requiredData = musicData.select('customer_id','product_id','star_rating','review_id','review_body').cache()
#The required coloumns for assignment

VBox()

In [56]:
print("Total number of rows")
requiredData.count() # count the total number of rows 

VBox()

Total number of rows
4751577

In [57]:
print("Total number of distinct reviews")
requiredData.select('review_id').distinct().count() # It will find the unique review ID and counting them.

VBox()

Total number of distinct reviews
4751577

In [58]:
print("Result after removing all the null value from review_id")
filteredreview = requiredData.filter(requiredData.review_id.isNotNull())
filteredreview.count() #cheking wheather there is a difference in the value or is there any missing value.

VBox()

Result after removing all the null value from review_id
4751577

In [59]:
print("The number of unique users")
requiredData.select('customer_id').distinct().count() 

VBox()

The number of unique users
1940732

In [60]:
print("Result after removing all the null values from product_id")
filterproduct = requiredData.filter(requiredData.product_id.isNotNull())
filterproduct.count()

VBox()

Result after removing all the null values from product_id
4751577

In [61]:
print("The number of unique products")
requiredData.select('product_id').distinct().count()

VBox()

The number of unique products
782326

In [62]:
largestNumberOfReviewsPubByUsers = requiredData.groupBy("customer_id").count().sort(col("count").desc()).cache()
#Here we are grouping all the customers with their ID then counting them by creating a count coulumn next to 
# customer ID and sorting them in descending order

VBox()

In [63]:
print("Top 10 products ranked by the number of review published")
topTenReviewUsers = largestNumberOfReviewsPubByUsers.limit(10)
topTenReviewUsers.show(10)

VBox()

Top 10 products ranked by the number of review published
+-----------+-----+
|customer_id|count|
+-----------+-----+
|   50736950| 7168|
|   38214553| 5412|
|   51184997| 5369|
|   18116317| 4222|
|   23267387| 4023|
|   50345651| 3793|
|   14539589| 2896|
|   15725862| 2842|
|   19380211| 2592|
|   20018062| 2568|
+-----------+-----+

In [64]:
print("The largest number of reviews published by a single user")
topTenReviewUsers.show(1)

VBox()

The largest number of reviews published by a single user
+-----------+-----+
|customer_id|count|
+-----------+-----+
|   50736950| 7168|
+-----------+-----+
only showing top 1 row

In [65]:
medOfUser = largestNumberOfReviewsPubByUsers.approxQuantile("count", [0.5], 0)
# Here we are using the build in spark function called the approxQuantile
#which takes three parameters namely the column name, probability and relative error. 
#The value 0.5 gives the median. 0 relative error means exact quantile is computed. The return value is the approximate quantiles at the given probabilities.

VBox()

In [66]:
print("The Median number of reviews published by a year")
print(medOfUser)

VBox()

The Median number of reviews published by a year
[1.0]

In [67]:
larNoRevSingleProduct  = requiredData.groupBy("product_id").count().sort(col("count").desc()).cache()

VBox()

In [68]:
print("The top 10 products ranked by the number of reviews they have")
topTenProduct = larNoRevSingleProduct.limit(10)
topTenProduct.show(10)

VBox()

The top 10 products ranked by the number of reviews they have
+----------+-----+
|product_id|count|
+----------+-----+
|B00008OWZG| 3936|
|B0000AGWEC| 3326|
|B00MIA0KGY| 2699|
|B00NEJ7MMI| 2420|
|B000089RVX| 2376|
|B004EBT5CU| 2106|
|B0026P3G12| 2080|
|B00009PRZF| 2026|
|B00004XONN| 1901|
|B00006J6VG| 1793|
+----------+-----+

In [69]:
print("The largest number of reviews written for a single product")
topTenProduct.show(1)

VBox()

The largest number of reviews written for a single product
+----------+-----+
|product_id|count|
+----------+-----+
|B00008OWZG| 3936|
+----------+-----+
only showing top 1 row

In [70]:
medOfProduct = larNoRevSingleProduct.approxQuantile("count",[0.5],0)
# Here we are using the build in spark function called the approxQuantile
#which takes three parameters namely the column name, probability and relative error. 
#The value 0.5 gives the median. 0 relative error means exact quantile is computed. The return value is the approximate quantiles at the given probabilities.

VBox()

In [71]:
print("The median number of reviews a product has")
print(medOfProduct)

VBox()

The median number of reviews a product has
[2.0]

In [72]:
endtime = datetime.now()
print("Time taken for execution:")
print(endtime-starttime) #calculating the time difference and printing it out.

VBox()

Time taken for execution:
0:00:41.762354